In [1]:
# Importamos las librerías que necesitamos

# Librerías de extracción de datos
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup
import requests
import tqdm
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np
import datetime

# Requests
import requests

from time import sleep
import random

# Para trabajar con archivos dotenv y los tokens
# -----------------------------------------------------------------------
import os
import dotenv
dotenv.load_dotenv(dotenv_path="../src/.env")
# Para Importar desde rutas recursivas
# -----------------------------------------------------------------------
import sys
sys.path.append("../") #Para poder importar
from src.SupportVuelos import get_airport

# Aeropuertos
- MAD: 95565077
- OSL:
- NCE:

In [5]:
aeropuertos = ["madrid","niza","oslo"]
resultados = [get_airport(i) for i in aeropuertos]

In [11]:
[resultados[1][0]]

['NCE']

In [12]:
resultados
df = pd.DataFrame()
for i in range(0,len(resultados)):
    df_temp = pd.DataFrame({
        "skyId": [resultados[i][0]],
        "entityId": [resultados[i][1]],
        "flightPlaceType" : [resultados[i][2]],
        "localizedName" : [resultados[i][3]]})
    df = pd.concat([df,df_temp],ignore_index=True)

df

,skyId,entityId,flightPlaceType,localizedName
0,MAD,95565077,AIRPORT,Madrid-Barajas
1,NCE,95673860,AIRPORT,Niza
2,OSL,128667763,AIRPORT,Oslo-Gardermoen


In [13]:
df.to_csv("../datos/aeropuertos.csv")

In [16]:
df = pd.read_csv("../datos/aeropuertos.csv",index_col="Unnamed: 0")

In [17]:
df

,skyId,entityId,flightPlaceType,localizedName
0,MAD,95565077,AIRPORT,Madrid-Barajas
1,NCE,95673860,AIRPORT,Niza
2,OSL,128667763,AIRPORT,Oslo-Gardermoen


In [40]:
skyId = df["skyId"].tolist()
entityId = df["entityId"].tolist()
flightPlaceType = df["flightPlaceType"].tolist()
localizedName = df["localizedName"].tolist()
cabinClass = ["economy","premium_economy","business","first"]

In [43]:
dates = [["2024-10-24","2024-10-27"],
         ["2024-11-31","2024-11-03"],
         ["2024-11-07","2024-11-10"],
         ["2024-11-14","2024-11-17"],
         ["2024-11-21","2024-11-24"]]

In [44]:
querystrings = []

for i in range(1,len(entityId)):
    for date in dates:
        for clase in cabinClass:
            querystring = {
                "originSkyId":skyId[0],
                "destinationSkyId":"NCE",
                "originEntityId": entityId[0],
                "destinationEntityId": entityId[i],
                "date":date[0],
                "returnDate":date[1],
                "cabinClass":clase,
                "adults":"4",
                "sortBy":"best",
                "limit" : "10",
                "currency":"EUR",
                "market":"es-ES",
                "countryCode":"ES"
                }
                
            querystrings.append(querystring)

In [46]:
len(querystrings)

40

In [47]:
querystrings

[{'originSkyId': 'MAD',
  'destinationSkyId': 'NCE',
  'originEntityId': 95565077,
  'destinationEntityId': 95673860,
  'date': '2024-10-24',
  'returnDate': '2024-10-27',
  'cabinClass': 'economy',
  'adults': '4',
  'sortBy': 'best',
  'limit': '10',
  'currency': 'EUR',
  'market': 'es-ES',
  'countryCode': 'ES'},
 {'originSkyId': 'MAD',
  'destinationSkyId': 'NCE',
  'originEntityId': 95565077,
  'destinationEntityId': 95673860,
  'date': '2024-10-24',
  'returnDate': '2024-10-27',
  'cabinClass': 'premium_economy',
  'adults': '4',
  'sortBy': 'best',
  'limit': '10',
  'currency': 'EUR',
  'market': 'es-ES',
  'countryCode': 'ES'},
 {'originSkyId': 'MAD',
  'destinationSkyId': 'NCE',
  'originEntityId': 95565077,
  'destinationEntityId': 95673860,
  'date': '2024-10-24',
  'returnDate': '2024-10-27',
  'cabinClass': 'business',
  'adults': '4',
  'sortBy': 'best',
  'limit': '10',
  'currency': 'EUR',
  'market': 'es-ES',
  'countryCode': 'ES'},
 {'originSkyId': 'MAD',
  'destina